In [4]:
import math

from keras.layers import *
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
 
columns = ['Speed','m0c','m1c','acc','lat','lon']

boat_curved = pd.read_csv("Data/Anomalous_boat_data.csv")
boat_curved= boat_curved.drop(columns=["Unnamed: 0", "heading"])    
boat_curved['degrees'][0] = 250
boat_curved= boat_curved[:6600]
scaler = StandardScaler()
anomalous_data = scaler.fit_transform(boat_curved.drop(columns=['degrees']))
anomalous_df = pd.DataFrame(anomalous_data, columns=columns)

def add_noise(st_devs):
    data = np.ones(shape=(6600,6))
    for i in range(6600):
        for j, value in enumerate(columns):
            v = anomalous_data[i][j]
            noise = np.random.normal(v, st_devs[j])
            data[i][j] = v + noise
    return data

columns_new = ['Speed', 'm0c', 'm1c', 'acc', 'lat', 'lon', 'sin', 'cos']

devs = [0.5, 1, 1, 2, 0.03, 0.03]
noisy_data = add_noise(devs)
zeros = np.zeros(shape=(6600,8))
zeros[:noisy_data.shape[0], :noisy_data.shape[1]] = noisy_data
noisy_data = zeros
noisy_df = pd.DataFrame(noisy_data, columns=columns_new)



id_list = []
def sin_cos():
    sins = []
    cosins = []
    for i, degree in enumerate(boat_curved["degrees"]):
        value = degree + np.random.normal(0, 2)
        sin = np.sin(np.deg2rad(value))
        cos = np.cos(np.deg2rad(value))
        id_list.append(sin**2+cos**2)
        sins.append(sin)
        cosins.append(cos)
    return sins, cosins    

sin,cos = sin_cos()
noisy_df['sin'] = sin
noisy_df['cos'] = cos

lenght = 1
for i in range(lenght):
    new_data = pd.DataFrame(add_noise(devs), columns=columns)
    sin, cos = sin_cos()
    new_data['sin'] = sin
    new_data['cos'] = cos
    noisy_df = noisy_df.append(new_data) 
# 
# plt.plot(noisy_df['sin'][6620:13240])
# plt.plot(noisy_df['cos'][6620:13240])
# plt.title("Sins-Cosins")
# plt.show()
# 
# plt.plot(id_list)
# plt.title("IDLIST")
# plt.show()
# 
# plt.plot(boat_curved['degrees'][:6620])
# plt.title("DEGREES")
# plt.show()


print(noisy_df)



         Speed       m0c       m1c       acc       lat       lon       sin  \
0     4.727042  1.211135  0.319713  0.963110 -3.494182  0.893308 -0.931652   
1     5.996897 -0.956716  1.968090  1.993158 -3.525949  0.831522 -0.947293   
2     6.931250 -2.101735  1.150638 -5.347093 -3.467695  0.782439 -0.907227   
3     4.988403  0.954801  0.482011 -1.175013 -3.443318  0.787395 -0.919837   
4     5.268159  0.967480 -2.293241 -0.610502 -3.429674  0.795345 -0.922596   
...        ...       ...       ...       ...       ...       ...       ...   
6595 -0.518240 -0.720287 -1.202694  1.040152 -2.948955  4.345733  0.980194   
6596  0.461944 -1.116251  2.872071  0.836972 -3.006043  4.390318  0.990572   
6597  0.230013 -0.574882 -2.013143 -3.593722 -2.985166  4.421186  0.993796   
6598 -1.110622  0.585431  0.182326 -3.274700 -2.973222  4.457709  0.999497   
6599 -1.537429 -0.291918 -0.333150  2.034964 -3.016558  4.439984  0.983534   

           cos  
0    -0.363352  
1    -0.320368  
2    -0.4206

In [5]:
noisy_df.to_csv("Data/Boat_anomalous_validation.csv")